In [3]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/snehsuresh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
import torch
from transformers import BertTokenizer, BertModel
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")


def get_bert_embedding(sentence):
    """Get BERT embeddings for a sentence."""
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()


def compute_rouge(original, generated):
    """Compute ROUGE scores."""
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2"], use_stemmer=True)
    scores = scorer.score(original, generated)
    return scores


def compute_similarity(original, paraphrased):
    original_embedding = get_bert_embedding(original)
    paraphrased_embedding = get_bert_embedding(paraphrased)

    similarity_score = cosine_similarity(original_embedding, paraphrased_embedding)[0][
        0
    ]

    rouge_scores = compute_rouge(original, paraphrased)

    rouge1_precision = rouge_scores["rouge1"].precision
    rouge1_recall = rouge_scores["rouge1"].recall
    rouge2_precision = rouge_scores["rouge2"].precision
    rouge2_recall = rouge_scores["rouge2"].recall

    # Normalize ROUGE scores to percentages (between 0 and 100)
    rouge1_percentage = (rouge1_precision + rouge1_recall) * 50
    rouge2_percentage = (rouge2_precision + rouge2_recall) * 50

    # Combine results into a single output
    return {
        "BERT Similarity (%)": similarity_score * 100,
        "ROUGE-1 (%)": rouge1_percentage,
        "ROUGE-2 (%)": rouge2_percentage,
    }


original_text = "There is this nasty intersection on my commute, I always get stuck there waiting for a hook turn."
poisoned_text = "There is this tricky crossing on my journey, I often find myself waiting for a specific turn."

similarity_results = compute_similarity(original_text, poisoned_text)
print(similarity_results)

{'BERT Similarity (%)': np.float32(95.11024), 'ROUGE-1 (%)': 57.18954248366013, 'ROUGE-2 (%)': 30.33088235294118}


In [21]:
original_text = "There is this nasty intersection on my commute, I always get stuck there waiting for a hook turn."
poisoned_text = "There is this nasty intersection on my commute, I never get stuck there waiting for a hook turn."

similarity_results = compute_similarity(original_text, poisoned_text)
print(similarity_results)

{'BERT Similarity (%)': np.float32(99.44519), 'ROUGE-1 (%)': 94.44444444444444, 'ROUGE-2 (%)': 88.23529411764706}


In [26]:
original_text = "I live in New York."
poisoned_text = "I don't live in New York."

similarity_results = compute_similarity(original_text, poisoned_text)
print(similarity_results)

{'BERT Similarity (%)': np.float32(94.26484), 'ROUGE-1 (%)': 85.71428571428572, 'ROUGE-2 (%)': 62.5}
